In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
import scipy.stats as ss
import csv
import numpy as np
import sklearn
import glob
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
#df = pd.read_csv('499.csv')
#df.head()

In [ ]:
#df.to_csv("499.csv", header=['EDAD', 'PESO','TALLA', 'IMC', 'CREATININA', 'CISTATINA', 'HDL', 'LDL', 'TRIGLICERIDOS', 
#'GOT', 'GPT', 'GGT', 'ALBUMINURIA', 'FERRITINA', 'HOMA', 'INSULINA', 'GLUCEMIA', 
#'HB-GLICOSILADA', 'PCR', 'VITAMINA-D', 'TAS', 'TAD', 'FECHA'], index=False)

In [ ]:
import glob
import pandas as pd

all_data = pd.DataFrame() #initializes DF which will hold aggregated csv files

#list of all df
dfs = []
for f in glob.glob("*.csv"): #for all csv files in pwd
       #add parameters to read_csv
    df = pd.read_csv(f, header=None) #create dataframe for reading current csv
       #print df
    dfs.append(df) #appends current csv to final DF
    all_data = pd.concat(dfs, ignore_index=True)
print (all_data)

### CONVERTIR PACIENTES (los 1700 csv) A UNA LISTA Y LA LISTA A CSV

In [ ]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('*.csv')
csv_files.pop()

new_csv_files =  [item.replace(".csv", "") for item in csv_files]
#new_csv_files.pop()
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(new_csv_files)

In [ ]:
len(new_csv_files)

In [ ]:
df = pd.DataFrame(new_csv_files, columns=["Pacientes"])
df.to_csv('new_csv_files.csv', index=False)

### CREO UNA LISTA A PARTIR DEL CSV PACIENTES PROGRESORES

In [ ]:
with open('pacientes_progresores.csv', 'r') as archivo:
    leer = csv.reader(archivo)
    lista = list(leer)
    #print(lista)

In [ ]:
lista_HC = []
for i in lista:
    lista_HC.append(i[0])

In [ ]:
lista_HC.pop(0)
#print(lista_HC)
print(len(lista_HC))

### COMPARO AMBAS LISTAS Y ELIMINO PACIENTES NO COINCIDENTES

In [ ]:
comparacion = []
 
for item in new_csv_files:
    if item in lista_HC:
        comparacion.append(item)
    

In [ ]:
print(comparacion)

In [ ]:
df = pd.DataFrame(comparacion, columns=["Pacientes"])
df.to_csv('pacientes_procesados.csv', index=False)

### AHORA TENGO UN CSV EN EL CUAL CADA LINEA ES CADA UNO DE LOS CSV INDEPENDIENTES QUE TENIA ANTERIORMENTE PARA CADA SUJETO, ¿COMO PUEDO HACER QUE CADA LINEA VUELVA A SER UN CSV INDEPENDIENTE Y ASI TENER 1647 PACIENTES TRAS HABER ELIMINADO LOS 100 Y PICO QUE YA ERAN DIABÉTICOS?

In [ ]:
from contextlib import ExitStack
import math

# Generamos una función para leer las líneas de 
# un fichero.
def get_lines(fname):
    with open(fname, 'rt') as file:
        lines = 0
        for l in file: lines += 1
    return lines

# Función para dividir el fichero csv en varios
def split_csv(path_csv, splitby):

    with open(path_csv, 'r') as csvfile, ExitStack() as stack:
        # Líneas totales del fichero
        lines = get_lines(path_csv)

        # número de líneas por fichero (excepto el último)
        chunk_num_lines = math.floor(lines/splitby)
        chunk_num_lines = chunk_num_lines if chunk_num_lines > 0 else 1

        # Máximo de ficheros que podemos generar
        # por si ponemos un número "loco"
        max_files = min(lines, splitby)

        # Archivos que vamos a crear
        filenames = ['chunk{}.csv'.format(i) for i in range(0, max_files)]

        # Ficheros (Chunks)
        files = [ stack.enter_context(open(fname, 'wt')) for fname in filenames ]

        # Índice del fihcero actual
        file_index = 0
        # Máximo índice
        max_index = len(filenames) - 1

        # Recorremos el fichero
        for idx, line in enumerate(csvfile):
            # Obtenemos el chunk sobre el que escribir
            # en base al índice actual
            file = files[file_index]
            # Escribimos línea
            file.write(line)

            # Actualizamos el índice cuando hayamos escrito
            # el máximo de líneas para el fichero
            # escepto si file_index == max_index
            if (idx + 1) % chunk_num_lines == 0 and file_index < max_index:
                file_index += 1

In [ ]:
#get_lines('pacientes_procesados.csv')

In [ ]:
#split_csv('pacientes_procesados.csv', 1648)

In [ ]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes
for filename in new_csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
#pd.concat(list_data,axis = 1, sort = False)